In [30]:
from IPython.utils import io
import csv
with io.capture_output() as captured:  
   %run ../../2-FeatureExtraction/ContextualEmbeddings/ContextualEmbeddings.ipynb import x, X, y1, y2, Y1, Y2, Features, Preprocesss

In [18]:
from torch import nn
from torch.optim import Adam
import torch
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import re
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import numpy as np

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu"); print(device)

cpu


#### Loading the Data

In [20]:
train_data = TensorDataset(torch.from_numpy(x), torch.from_numpy(y1))
valid_data = TensorDataset(torch.from_numpy(X), torch.from_numpy(Y1))

batch_size = 128

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=1024)

#### Implementing the Model Head

In [21]:
class ContextualClassifier(nn.Module):

  def __init__(self, layer_1=768, layer_2=512, output=3):
    super(ContextualClassifier, self).__init__()
    self.linear_1 = nn.Linear(layer_1, layer_2)
    self.linear_2 = nn.Linear(layer_2, output)
    self.relu = nn.ReLU()

  def forward(self, contextualized_embeddings):
    linear_1_output = self.linear_1(contextualized_embeddings)
    linear_2_output = self.linear_2(self.relu(linear_1_output))
    output_layer = self.relu(linear_2_output)
    return output_layer


#### Initializing the Model

In [22]:
EPOCHS = 300
LR = 3e-6
N1 = 768
N2 = 512
WeightedLoss = False
w = torch.tensor(np.load("../../Dataset/w1.npy").astype(np.float32)).to(device)
w = torch.tensor([0.3, 0.25, 0.7])

ModelInfo = {
   "Model": "Arabert-1",
   "batch_size": batch_size,
   "Hidden Dimension 1":N1,
   "Hidden Dimension 2": N2,
   "Learning Rate": LR,
   "Number of Epochs": EPOCHS,
   "Weighted Loss": WeightedLoss,
   "Weights": w
}

model = ContextualClassifier(layer_1=N1, layer_2=N2).to(device)

In [23]:
if WeightedLoss:
    criterion = nn.CrossEntropyLoss(weight=w).to(device)
else:
    criterion = nn.CrossEntropyLoss().to(device)
    
optimizer = Adam(model.parameters(), lr=LR)

#### Training the Model

In [24]:
for epoch_num in tqdm(range(EPOCHS)):
  for train_input, train_label in train_loader:
    model.zero_grad()

    train_input = train_input.to(device)
    train_label = train_label.to(device)
    output = model(train_input)
    output = output.squeeze(1)
    batch_loss = criterion(output, train_label)

    batch_loss.backward()
    optimizer.step()

100%|██████████| 300/300 [04:21<00:00,  1.15it/s]


#### Validating the Model

In [31]:
output_labels = []

with torch.no_grad():
  for valid_input, valid_label in tqdm(valid_loader):
    valid_input = valid_input.to(device)
    valid_label = valid_label.to(device)
    output = model(valid_input).cpu()
    output = output.squeeze(1)
    output_labels.extend(output.argmax(dim=1).tolist())

F1 = f1_score(Y1, output_labels, average='macro')
Report = classification_report(Y1, output_labels, zero_division=0, output_dict=True)
print(classification_report(Y1, output_labels, zero_division=0))

100%|██████████| 1/1 [00:00<00:00, 15.31it/s]

              precision    recall  f1-score   support

           0       0.13      0.56      0.21       126
           1       0.80      0.45      0.58       804
           2       0.08      0.01      0.02        70

    accuracy                           0.43      1000
   macro avg       0.34      0.34      0.27      1000
weighted avg       0.67      0.43      0.49      1000



#### Saving the Model

In [32]:
with open('runs.csv', 'a') as f:  
      run_info = {**Preprocessing, **Features, **ModelInfo, "acc":Report["accuracy"],"BF1": Report["macro avg"]["f1-score"], "WF1": Report["weighted avg"]["f1-score"] }
      w = csv.DictWriter(f, run_info.keys())
      w.writeheader()
      w.writerow(run_info)